In [3]:
import os
os.chdir('/home/rafalb/work/molecules/chemicalSmilesSpace/src')

In [13]:
import argparse
#from tabulate import tabulate
from pathlib import Path
import predictiveModel
import predictSmiles
import numpy as np
import random

In [27]:
predictiveModel = predictiveModel.unpicklePredictiveModel('predictiveModel646432_20191027.pckl')

In [65]:
nCharInSmiles = predictiveModel['nCharSmiles']
nCharSet = predictiveModel['nCharSet']
nStatic = predictiveModel['nStatic']
weightsFile = predictiveModel['model']
char2indices = predictiveModel['char2indices']
indices2char = predictiveModel['indices2char']
nLatent = predictiveModel['nLatent']
print(weightsFile)

staticDynmic_6464646432_70000_512.h5


In [12]:
decoder = predictSmiles.loadDecoder(nCharInSmiles, nCharSet, nStatic, weightsFile)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputStatic (InputLayer)        (None, 3)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           256         inputStatic[0][0]                
__________________________________________________________________________________________________
inputDynamic (InputLayer)       (None, 60, 27)       0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           4160        dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (D

In [75]:
latentVector = [random.random() for iii in range(nLatent)]
latentVector = np.array(latentVector).reshape(-1, 32)
latentVector *= 10
print(latentVector)
    
prediction = decoder.predict(latentVector)[0].reshape(nCharInSmiles, nCharSet)

[[7.91632481 1.05000931 5.05945471 9.04675349 4.88309245 5.00058411
  6.76805375 0.80003954 7.26793153 9.79356426 7.17791875 9.27460229
  3.83158637 2.26007372 4.68711324 4.63721871 3.44176021 3.95207417
  7.69285749 4.66096233 4.86081444 4.27301981 9.82128357 4.89456253
  8.13946964 4.75840521 4.14685096 3.68789454 3.50253204 3.62568268
  7.75007779 7.49465428]]


In [76]:
smiles = []
for iii in range(nCharInSmiles):
    smiles.append(indices2char[np.argmax(prediction[iii, :])])
smiles


['C',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ']

In [60]:
smiles = []
for iii in range(nCharInSmiles):
    smiles.append(indices2char[np.argmax(prediction[iii, :])])
smiles


['C',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ']